In [ ]:
import json
import os
import torch
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import scipy
import neptune
import torch.nn as nn
from tqdm import tqdm
from torch.nn.functional import cross_entropy
from torchsummary import summary
from utils import scale_to_unit_range
from DataLoader import EuroSAT
from engine import train_one_epoch , test_one_epoch
from torchvision import datasets , transforms
from torchvision.transforms import ToTensor


In [ ]:
# Hyperparameters
transform = transforms.Compose([
    ToTensor() , 
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) ,    
])
BATCH_SIZE = 8 
LR = 0.001
Epochs = 25 
device = 'cuda' if torch.cuda.is_available() else "cpu"
device


Dataset Prep

In [ ]:
train_csv = pd.read_csv("EuroSAT/train.csv" , index_col = 0)
test_csv = pd.read_csv("EuroSAT/test.csv" , index_col = 0)
val_csv = pd.read_csv("EuroSAT/validation.csv" , index_col = 0)

train_csv = train_csv.sort_values(axis = 0 , by = ['ClassName'])
test_csv = test_csv.sort_values(axis = 0 , by = ['ClassName'])
val_csv = val_csv.sort_values(axis = 0 , by = ['ClassName'])

In [ ]:
with open("EuroSAT/label_map.json" , 'r') as file:
    labels = json.load(file)
    class_names = list(labels.keys())
class_names 

In [ ]:
train_set = []
test_set = []
val_set = []
sets = [train_csv , val_csv , test_csv]

for i , set in enumerate(sets):
    if i == 0:
        for index, row in set.iterrows():
            train_set.append(list(row))
    elif i == 2:
        for index, row in set.iterrows():
            test_set.append(list(row))
            
    else: 
        for index, row in set.iterrows():
            val_set.append(list(row))           

In [ ]:
train = EuroSAT(parent_dir = "EuroSAT" , data = train_set , transform = transform)
val = EuroSAT(parent_dir = "EuroSAT" , data = val_set , transform = transform)
test = EuroSAT(parent_dir = "EuroSAT" , data = test_set , transform = transform)

len(train) , len(val) , len(test)

In [ ]:
train_loader = torch.utils.data.DataLoader(train , shuffle = True , batch_size=BATCH_SIZE)
val_loader = torch.utils.data.DataLoader(val , shuffle = True , batch_size=BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(test, shuffle = False , batch_size=BATCH_SIZE)

len(train_loader) , len(val_loader)  , len(test_loader)

Visualize some samples

In [ ]:
train_iter = iter(train_loader)
first_batch = next(train_iter)
images , labels  = first_batch

images.shape , labels.shape

In [ ]:
formatted_images  = []
for image in images:
    image = image.permute(1 , 2 , 0)
    image = scale_to_unit_range(image)
    image = image.numpy()
    formatted_images.append(image)
    
images[0].shape , formatted_images[0].shape

In [ ]:
nrows = 2
ncolumns = 4
fig, axs = plt.subplots(nrows, ncolumns, figsize=(15, 6))

# Flatten the axs array to simplify accessing individual subplots
axs = axs.flatten()

for i in range(len(images)):
    ax = axs[i]  # Access the individual subplot
    ax.imshow(formatted_images[i])  # Display the image
    ax.set_title(class_names[labels[i]])  # Set the title to the class name of the image
    ax.axis('off')  # Hide the axis

plt.show()

NN using Pytorch library

In [ ]:
from Networks.NN_1 import NN_1

model = NN_1()

In [ ]:
model = NN_1().to(device)
optimizer = torch.optim.Adam(model.parameters() , lr = LR)
cross_entropy = torch.nn.CrossEntropyLoss()
scheduler = "exponential"
if scheduler == "onecyclelr":
    lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=LR, steps_per_epoch=len(train_loader), epochs=Epochs, pct_start=0.2)
elif scheduler == "multi_step_lr":
    lr_drop_list = [4, 8]
    lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=lr_drop_list)
elif scheduler == "step_lr":
    step_size = 10
    gamma = 0.5
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = step_size , gamma = gamma)
else:
    gamma = 0.95
    lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer , gamma)
summary(model , input_size=( 3 , 64 , 64))

In [ ]:
out_dir = 'weights/NN_attempt_1/best_checkpoint.pth'
train_loss , val_loss , current_lr = train_one_epoch(model ,
                                     training_loader=train_loader ,
                                     validation_loader = val_loader ,
                                     optimizer=optimizer ,
                                     lr_scheduler = lr_scheduler , 
                                     epochs = Epochs , 
                                     loss_func = cross_entropy ,
                                     device = device ,
                                     out_dir = out_dir) 

In [ ]:
checkpoint = torch.load("weights/NN_attempt_1/best_checkpoint.pth")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

model.eval()
model.to(device)
test_loss = test_one_epoch(model = model , test_loader= test_loader , loss_func=cross_entropy , device = device)